In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense


In [2]:
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [3]:
path_to_data = 'ngp.csv' # путь к файлу, из которого берутся данные для обучения
target = 'price' # название взятой величины из файла
FIT_batch_size = 4 #13, 4 # размер bach при обучении/тестировании

In [4]:
# извлечь датасет из файла
def get_df(path, target_name):
    file = pd.read_csv(path)
    dframe = pd.DataFrame(list(reversed(file[target_name])))
    return dframe

In [5]:
# заполнить колонки и имена колонок для тестового датасета
def get_cols_and_names(df):
    input_t = 4
    output_t = 1
    
    cols = list()
    names = list()
    
    for i in range(input_t,0, -1):
        cols.append(df.shift(i))
        names.append(i)
    for i in range(0,output_t):
        cols.append(df.shift(-i))                    
        names.append(i+input_t+1)
    return cols,names

In [6]:
# разбиение данных на тестовые и тренировочные
def split(X,Y,factor):
    X_train=X[:factor]
    Y_train=Y[:factor]
    X_test=X[factor:]
    Y_test=Y[factor:]
    return X_train,Y_train,X_test,Y_test

In [7]:
# подготовить датасет из файла
def prepare_df(cols,names):
    n_df = pd.concat(cols, axis = 1)
    n_df.columns = names
    n_df.dropna(inplace = True)
    return n_df

In [8]:
#подготовить данные к разбиению на тестовые и тренировочные
def prep_data_4_1_time(cols, names, prepared_df):
    X = prepared_df[[4, 3, 2, 1]]
    Y = prepared_df[5]
    X = X / X.max()
    Y = np.asarray(Y)
    Y = Y / Y.max()
    
    factor = int(.80*X.shape[0])

    return split(X ,Y , factor)

In [9]:
# подготовка данных

# извлечь датасет из файла
df = get_df(path_to_data, target)

# заполнить колонки и имена колонок для тестового датасета
cols, names = get_cols_and_names(df)

# создать тестовый/тренировочный датасет
prepared_df = prepare_df(cols,names)

# создать тестовые/тренировочные данные из тестового датасета
prepared_data = prep_data_4_1_time(cols, names, prepared_df)

X_train,Y_train,X_test,Y_test = prepared_data

In [10]:
model = keras.models.load_model('NetWork_4_to_1')

In [21]:
#print(type(X_test))
#for i, row in X_test.iterrows():
 #   print(f"Index: {i}")
#    print(f"{row}\n")
print(X_test.shape)
print(Y_test.shape)

(215, 4)
(215,)


In [12]:
# тест модели

model.evaluate(X_test, Y_test, batch_size = FIT_batch_size)

54/54 [==============================] - 0s 278us/step - loss: 4.5868e-04


0.00045867630979046226

In [35]:
# вывести пару результатов работы
x = 0
for i, row in X_test.iterrows():
    if x > 1:
        a = pd.DataFrame([row])
        res = model.predict(a)  
        print((f"{x-1} Time series: %.4f " % row[4]) + ("%.4f " % row[3]) + ("%.4f " % row[2]) + ("%.4f " % row[1]))
        print("Network's output: %.4f" % res[0,0])
        print("Right answer: %.4f" % Y_test[x])
        print("Difference: %.4f" % abs(Y_test[x] - res[0,0]))
        print("------------------------------")
    x+=1
    if x > 5:
        break


1 Time series: 0.2291 0.2312 0.2409 0.2450 
Network's output: 0.2417
Right answer: 0.2512
Difference: 0.0095
------------------------------
2 Time series: 0.2312 0.2409 0.2450 0.2512 
Network's output: 0.2481
Right answer: 0.2491
Difference: 0.0011
------------------------------
3 Time series: 0.2409 0.2450 0.2512 0.2491 
Network's output: 0.2479
Right answer: 0.2560
Difference: 0.0082
------------------------------
4 Time series: 0.2450 0.2512 0.2491 0.2560 
Network's output: 0.2507
Right answer: 0.2450
Difference: 0.0057
------------------------------
